# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,21.73,97,100,3.60,US,1718806341
1,1,west bay,19.3667,-81.4167,30.03,80,97,6.86,KY,1718806341
2,2,revda,56.8010,59.9303,26.69,63,11,2.54,RU,1718806342
3,3,adamstown,-25.0660,-130.1015,22.24,86,100,10.12,PN,1718806342
4,4,waitangi,-43.9535,-176.5597,8.79,65,100,5.78,NZ,1718806342


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points("Lng","Lat",geo = True, tiles = "OSM",frame_width = 800,frame_height = 600,size = "Humidity",scale = 1, color = "City")

# Display the map
city_data_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_cleaned = ideal_weather.dropna()

# Display sample data
ideal_weather_cleaned

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,kailua-kona,19.6406,-155.9956,24.22,86,0,1.54,US,1718806284
130,130,todos santos,23.4500,-110.2167,21.39,61,0,3.90,MX,1718806365
159,159,arraial do cabo,-22.9661,-42.0278,26.92,65,0,3.60,BR,1718806370
250,250,nantucket,41.2835,-70.0995,25.06,84,0,4.12,US,1718806394
285,285,tsiombe,-25.3000,45.4833,23.32,63,0,3.56,MG,1718806401
323,323,kencong,-8.2833,113.3667,23.30,86,0,2.25,ID,1718806408
377,377,cabo san lucas,22.8909,-109.9124,24.61,70,0,2.06,MX,1718805805
446,446,cuatro cienegas de carranza,26.9833,-102.0833,25.19,69,0,2.15,MX,1718806431
504,504,sonoita,31.8500,-112.8333,25.84,26,0,2.46,MX,1718806446
562,562,santa rosalia,27.3167,-112.2833,24.90,23,0,4.04,MX,1718806457


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cleaned[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,kailua-kona,US,19.6406,-155.9956,86,
130,todos santos,MX,23.4500,-110.2167,61,
159,arraial do cabo,BR,-22.9661,-42.0278,65,
250,nantucket,US,41.2835,-70.0995,84,
285,tsiombe,MG,-25.3000,45.4833,63,
323,kencong,ID,-8.2833,113.3667,86,
377,cabo san lucas,MX,22.8909,-109.9124,70,
446,cuatro cienegas de carranza,MX,26.9833,-102.0833,69,
504,sonoita,MX,31.8500,-112.8333,26,
562,santa rosalia,MX,27.3167,-112.2833,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 25}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
todos santos - nearest hotel: Hotel Casa Tota
arraial do cabo - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
tsiombe - nearest hotel: Hôtel Littoral Tsihombe
kencong - nearest hotel: Grand Gumuk Mas Hotel
cabo san lucas - nearest hotel: Comfort Rooms
cuatro cienegas de carranza - nearest hotel: No hotel found
sonoita - nearest hotel: No hotel found
santa rosalia - nearest hotel: Sol y Mar


,City,Country,Lat,Lng,Humidity,Hotel Name
19,kailua-kona,US,19.6406,-155.9956,86,Kona Seaside Hotel
130,todos santos,MX,23.4500,-110.2167,61,Hotel Casa Tota
159,arraial do cabo,BR,-22.9661,-42.0278,65,No hotel found
250,nantucket,US,41.2835,-70.0995,84,No hotel found
285,tsiombe,MG,-25.3000,45.4833,63,Hôtel Littoral Tsihombe
323,kencong,ID,-8.2833,113.3667,86,Grand Gumuk Mas Hotel
377,cabo san lucas,MX,22.8909,-109.9124,70,Comfort Rooms
446,cuatro cienegas de carranza,MX,26.9833,-102.0833,69,No hotel found
504,sonoita,MX,31.8500,-112.8333,26,No hotel found
562,santa rosalia,MX,27.3167,-112.2833,23,Sol y Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", size="Humidity", color="City",scale=1, geo=True, tiles="OSM", frame_width=800, frame_height=600,hover_cols=['Country', 'Hotel Name'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)